# NCrystal data infrastructure and standard data library

In this notebook we will discuss the NCrystal data library of predefined materials, as well as the general infrastructure for such data.

## Preamble
Install dependencies and prepare plots. Feel free to edit as you wish:

In [ ]:
#By default we only do pip installs on Google Colab, but you
#can set the variable in the next line to True if you need it:
always_do_pip_installs = False
try:
    import google.colab as google_colab
except ModuleNotFoundError:
    google_colab=None#not on google colab
if always_do_pip_installs or google_colab:
    from importlib.util import find_spec as _fs
    if not _fs('NCrystal'):
        %pip -q install ncrystal ipympl numpy matplotlib
#enable inline and interactive matplotlib plots:
if google_colab:
    google_colab.output.enable_custom_widget_manager()
%matplotlib ipympl
import matplotlib
matplotlib.rcParams.update({"figure.autolayout": True})
#always import NCrystal:
import NCrystal as NC
assert NC.version_num >=  3009006, "too old NCrystal found"
NC.test() #< quick unit test that installation works

## The NCrystal standard data library
The first and foremost source, is of course materials pre-created by other scientists (be it NCrystal developers or your colleagues). Any complete NCrystal installation should include the standard data library ("stdlib") of NCMAT files. For the ~latest NCrystal release, it can be browsed online at the wiki: https://github.com/mctools/ncrystal/wiki/Data-library

So assuming you find the material (say, zirconia) you were looking for on that page, you can as usual proceed to have a look at it:

In [ ]:
import NCrystal.plot as ncplot
ncplot.plot_xsect('ZrO2_sg137_Zirconia.ncmat')#remember: room temperature by default

### Physical (on-disk) and virtual files (in-memory) data
Depending on your installation of NCrystal, the "files" in the standard data library might reside as physical files on-disk, or they might be "baked in" to the NCrystal binary library as virtual files. It should not matter to you. If you wish to get the raw content of a file, you can do so with the `createTextData` function (or via `nctool --extract` on the commandline):

In [ ]:
td = NC.createTextData('stdlib::ZrO2_sg137_Zirconia.ncmat')
# or this way, to prevent accidentally picking up a file you downloaded,
# edited, and left lying around in your working directory:
# td = NC.createTextData('stdlib::ZrO2_sg137_Zirconia.ncmat')

Note that the `stdlib::` part is optional. By specifying it, we avoid accidentally picking up a file of the same name that you might have previously downloaded, edited, and left lying around in your working directory. Anyway, what can we do with such a `TextData` object?:

In [ ]:
help(td)

In [ ]:
print(td.dataSourceName)
print(td.lastKnownOnDiskLocation)#will be None if file does not exist on-disk
print(td.dataType)
print('-- contents: --')
print(td.rawData)

Similarly, when you yourself need to add a new NCMAT file, you do not actually have to write a physical file into your filesystem (unless you need the file to persist after your current process is done of course). You can instead simply register the associated data as a physical file in the currently running process:

In [ ]:
a_string_with_ncmat_data="""NCMAT v7
#Don't use this material for anything
@DENSITY
  1.2345 g_per_cm3
@DYNINFO
  element  C
  fraction 1
  type     freegas
"""
NC.registerInMemoryFileData('silly_carbon.ncmat',a_string_with_ncmat_data)

In [ ]:
print(NC.createTextData('silly_carbon.ncmat').rawData)

In [ ]:
ncplot.plot_xsect('silly_carbon.ncmat;temp=200K',mode='ekin')

Of course, if all you wanted was to quickly plot the cross sections, you didn't need to register the file and invent a name for it. You could either do:

In [ ]:
mat_silly=NC.load(a_string_with_ncmat_data)
mat_silly.plot()

Or, if you really didn't need to use the material for anything else than this plot:

In [ ]:
ncplot.plot_xsect(a_string_with_ncmat_data)

### NCrystal data source infrastructure
As we are starting to see, the NCrystal infrastructure which serves up data files based on "filenames" in cfg-strings is rather flexible. The `NCrystal.datasrc` module contain many functions which can be used to fine-tune this. For instance, if you keep your own edition to a data-library in some local folder, you can add that folder to the NCrystal search path. You can either do this by setting the `NCRYSTAL_DATA_PATH` variable (*before* loading NCrystal!), which can contain multiple directories separated by semi-colons in the usual unix fashion. Or you can add a directory to this search path dynamically:

In [ ]:
import pathlib
extra_data_dir = pathlib.Path('./myextradatafiles/')
extra_data_dir.mkdir(exist_ok=True)
( extra_data_dir / 'my_extra_material.ncmat' ).write_text(a_string_with_ncmat_data)

In [ ]:
!ls ./myextradatafiles

In [ ]:
import NCrystal.datasrc as ncdatasrc
ncdatasrc.addCustomSearchDirectory(extra_data_dir)
td = NC.createTextData('my_extra_material.ncmat')
print(td.dataSourceName)
print(td.lastKnownOnDiskLocation)#will be None if file is virtual (but it won't be in this case)
print(td.dataType)
print('-- contents: --')
print(td.rawData)

We have already seen how you can browse available data files via the command line with `nctool -b` or `nctool --browse`, but you can of course also do so from Python:

In [ ]:
NC.browseFiles(dump=True)

## Atom data
NCrystal obviously contains an internal database of scattering lengths, cross sections and masses, of a large number of natural elements and specific isotopes. You can access this database directly if needed:

In [ ]:
NC.atomDB('Al')

In [ ]:
NC.atomDB('He3')

You can access the data programatically, and as always you can find inline help:

In [ ]:
data_H = NC.atomDB('H')
data_D = NC.atomDB('D')
print(data_H.coherentScatLen())
print(data_D.coherentScatLen())
help(data_H)

Here is how to iterate through the entire database:

In [ ]:
for e in NC.iterateAtomDB():
    print(e)

It is always possible to override some of these values for a particular material, or even to provide values for elements or isotopes that might be missing. This can be done either in the `@ATOMDB` section of NCMAT data (cf. https://github.com/mctools/ncrystal/wiki/NCMAT-format), or through the `atomdb` cfg-string variable (cf. https://github.com/mctools/ncrystal/wiki/CfgRefDoc). We will return to this subject when discussing the `NCMATComposer` in another Notebook.

### Gas mixtures and other on-demand NCMAT data for simple materials
As a nice side-effect from having the flexible data source infrastructure, NCrystal also contains several plugins providing "quick materials": Materials so simple, that they can be expressed in a "filename". Rather than locating a real file, the plugin handling the request will analyse the intent expressed in the "filename", and dynamically compose the appropriate NCMAT data to reflect this request. The best example of this is the `"gasmix"` plugin, handling gas mixtures:

* `"gasmix::0.72xCO2+0.28xAr/massfractions/1.5atm/250K"`
* `"gasmix::0.7xCO2+0.3xAr/0.001relhumidity"`
* `"gasmix::0.7xCO2+0.3xAr/1.5atm/250K"`
* `"gasmix::BF3/2atm/25C/B_is_0.95_B10_0.05_B11"`
* `"gasmix::CO2"`
* `"gasmix::He/1.64kgm3"`
* `"gasmix::He/10bar"`
* `"gasmix::air"`
* `"gasmix::air/-10C/0.8atm/0.30relhumidity"`

Hopefully, you can guess what each line above would give you! The only thing to be aware about though, is that unless `/massfractions` are explicitly requested as in the first example above, the mixture is going to be by-volume (or by-mole of molecules which is the same thing in this case).

You can of course always check the result by looking at the NCMAT data created in response to the request (cf. https://github.com/mctools/ncrystal/wiki/NCMAT-format). You can also load and inspect the material. First the NCMAT data:

In [ ]:
print(NC.createTextData("gasmix::0.7xCO2+0.3xAr/25C/0.001relhumidity").rawData)

Or look at the loaded `Info` object:

In [ ]:
NC.createInfo("gasmix::0.7xCO2+0.3xAr/25C/0.001relhumidity").dump()

Or plot the cross sections:

In [ ]:
ncplot.plot_xsect("gasmix::0.7xCO2+0.3xAr/25C/0.001relhumidity")

One thing you might have noticed about the dumped NCMAT data above is the section:
```
@TEMPERATURE
  298.15
```
It means that NCrystal will refuse to let the user change the temperature of this material further, which is a safeguard against someone trying to change the temperature like this (which would give a material with the wrong density!):

In [ ]:
try:
    NC.load("gasmix::0.7xCO2+0.3xAr/25C/0.001relhumidity;temp=10K")
except NC.NCBadInput as e:
    print("NCBadInput ERROR: %s"%e)

The right way to modify the temperature is inside the "filename" part (changing `25C`->`10K`):

In [ ]:
NC.load("gasmix::0.7xCO2+0.3xAr/10K/0.001relhumidity")


Two other "quick factories" exists, which can be useful in the case where a non-gaseous material needs to be included in a simulation, but where the exact material *structure* is not important (or, not known!):

* `"freegas::Ar/2.5e-5perAa3"`
* `"freegas::CF4/3.72kgm3"`
* `"freegas::CO2/1.98kgm3"`
* `"freegas::He/0.17kgm3/He_is_He3"`
* `"solid::Al2O3/4gcm3"`
* `"solid::B4C/2.52gcm3/B_is_0.95_B10_0.05_B11"`
* `"solid::CH2/1gcm3"`
* `"solid::Gd2O3/7.07gcm3"`
* `"solid::Al2O3/4gcm3/TDebye750K_Al/TDebye1000K_O"`
* `"solid::Al2O3/4gcm3/TDebye900K"`


These materials will get the density and composition which can be inferred from the strings. In the case of `freegas::` materials, all atoms will be modelled as independent free gas atoms. In a way this is no different to how materials are mostly modelled in OpenMC/Geant/MCNP/..., but it is useful to have such simple materials in NCrystal as well - not the least for when NCrystal is used standalone or in applications like McStas where there is otherwise no universal concept of a "base material".

The `solid::` materials are very similar, but they differ in that their simplistic dynamics will be appropriate for bound atoms rather than free atoms. Such a feature was easy enough to add in NCrystal, since we can generate all dynamics of an amorphous material based on a 1D phonon DOS curve, and we simply use a simplistic phonon DOS curve as input (the Debye model).